In [1]:
from collections import Counter
from pathlib import Path
import os
import numpy as np
import re
from exception import CustomException
import sys
import string
import nltk
# nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
# nltk.download('stopwords')
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
# nltk.download('wordnet')
from nltk.corpus import wordnet
import unicodedata
import html
stop_words = stopwords.words('english')

In [16]:
import re
re1 = re.compile(r'  +')
import html

def remove_special_chars(text):
    try:
        re1 = re.compile(r'  +')
        x1 = text.lower().replace('#39;', "'").replace('amp;', '&').replace('#146;', "'").replace(
            'nbsp;', ' ').replace('#36;', '$').replace('\\n', "\n").replace('quot;', "'").replace(
            '<br />', "\n").replace('\\"', '"').replace('<unk>', 'u_n').replace(' @.@ ', '.').replace(
            ' @-@ ', '-').replace('\\', ' \\ ')
        return re1.sub(' ', html.unescape(x1))
    except Exception as e:
        raise CustomException(e, sys)


def remove_non_ascii(text):
    try:
        """Remove non-ASCII characters from list of tokenized words"""
        return unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    except Exception as e:
        raise CustomException(e, sys)


def to_lowercase(text):
    try:
        return text.lower()
    except Exception as e:
        raise CustomException(e, sys)



def remove_punctuation(text):
    try:
        """Remove punctuation from list of tokenized words"""
        translator = str.maketrans('', '', string.punctuation)
        return text.translate(translator)
    except Exception as e:
        raise CustomException(e, sys)


def replace_numbers(text):
    try:
        """Replace all interger occurrences in list of tokenized words with textual representation"""
        return re.sub(r'\d+', '', text)
    except Exception as e:
        raise CustomException(e, sys)


def remove_whitespaces(text):
    try:
        return text.strip()
    except Exception as e:
        raise CustomException(e, sys)


def remove_stopwords(words, stop_words):
    try:
        """
        :param words:
        :type words:
        :param stop_words: from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
        or
        from spacy.lang.en.stop_words import STOP_WORDS
        :type stop_words:
        :return:
        :rtype:
        """
        return [word for word in words if word not in stop_words]
    except Exception as e:
        raise CustomException(e, sys)

def stem_words(words):
    try:
        """Stem words in text"""
        stemmer = PorterStemmer()
        return [stemmer.stem(word) for word in words]
    except Exception as e:
        raise CustomException(e, sys)

def lemmatize_words(words):
    try:
        """Lemmatize words in text"""

        lemmatizer = WordNetLemmatizer()
        return [lemmatizer.lemmatize(word) for word in words]
    except Exception as e:
        raise CustomException(e, sys)

def lemmatize_verbs(words):
    try:
        """Lemmatize verbs in text"""
        lemmatizer = WordNetLemmatizer()
        return ' '.join([lemmatizer.lemmatize(word, pos='v') for word in words])
    except Exception as e:
            raise CustomException(e, sys)


def text2words(text):
    try:
        return text.split()
    except Exception as e:
        raise CustomException(e, sys)

def normalize_text(text):
    try:
        text = remove_special_chars(text)
        text = remove_non_ascii(text)
        text = remove_punctuation(text)
        text = to_lowercase(text)
        text = replace_numbers(text)
        words = text2words(text)
        words = remove_stopwords(words, stop_words)
        words = lemmatize_words(words)
        words = lemmatize_verbs(words)

        return ''.join(words)
    except Exception as e:
        raise CustomException(e, sys)

In [27]:
import pandas as pd  
messages = pd.read_csv('SMSSpamCollection.txt', sep='\t', names = ['label', 'messages'])
messages

,label,messages
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [19]:
normalized_messages = []
for message in messages['messages']:
    normalized_messages.append(normalize_text(message))

In [55]:
normalized_messages
X = normalized_messages

## Create BOW

In [23]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features= 2500, ngram_range=(1,2))

In [31]:
# my independant feature
x = cv.fit_transform(normalized_messages)
y = messages[['label']]

,label
0,ham
1,ham
2,spam
3,ham
4,ham
...,...
5567,spam
5568,ham
5569,ham
5570,ham


In [ ]:
## output features 
new_y = pd.get_dummies(messages['label'],dtype=int).iloc[:,1].values

In [44]:
## Train Test Split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(x,new_y,test_size=0.20)

In [45]:
from sklearn.naive_bayes import MultinomialNB


In [46]:
spam_detect_model=MultinomialNB().fit(X_train,y_train)


In [47]:
y_pred=spam_detect_model.predict(X_test)


In [48]:
from sklearn.metrics import accuracy_score,classification_report


In [49]:
accuracy_score(y_test,y_pred)


0.9775784753363229

In [50]:

from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.98      0.99      0.99       963
           1       0.94      0.89      0.92       152

    accuracy                           0.98      1115
   macro avg       0.96      0.94      0.95      1115
weighted avg       0.98      0.98      0.98      1115



In [56]:
## Train Test Split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.20)

In [57]:
from sklearn.feature_extraction.text import TfidfVectorizer
tv=TfidfVectorizer(max_features=2500,ngram_range=(1,2))
X_train=tv.fit_transform(X_train)
X_test=tv.transform(X_test)

In [58]:
from sklearn.naive_bayes import MultinomialNB
spam_tfidf_model = MultinomialNB().fit(X_train, y_train)

In [59]:
y_pred=spam_tfidf_model.predict(X_test)


In [60]:

score=accuracy_score(y_test,y_pred)
print(score)

0.9713004484304932


In [61]:
from sklearn.metrics import classification_report
print(classification_report(y_pred,y_test))

              precision    recall  f1-score   support

         ham       1.00      0.97      0.98      1002
        spam       0.78      0.99      0.88       113

    accuracy                           0.97      1115
   macro avg       0.89      0.98      0.93      1115
weighted avg       0.98      0.97      0.97      1115



In [ ]:
! 